## Exercises:
* ##### E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
* ##### E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
* ##### E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
* ##### E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
* ##### E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
* ##### E06: meta-exercise! Think of a fun/interesting exercise and complete it.

### E01

In [1]:
words = ['..'+line.split(',')[1].lower()+'.' for line in open("data/NationalNames.csv", 'r').read().splitlines()][1:]

In [2]:
for i in range(5):
    word = words[i]
    print(word)
    trigrams = [(word[i:i + 2], word[i + 2]) for i in range(len(word) - 2)]
    print(f"Trigrams: {str(trigrams)}\n-----------")

..mary.
Trigrams: [('..', 'm'), ('.m', 'a'), ('ma', 'r'), ('ar', 'y'), ('ry', '.')]
-----------
..anna.
Trigrams: [('..', 'a'), ('.a', 'n'), ('an', 'n'), ('nn', 'a'), ('na', '.')]
-----------
..emma.
Trigrams: [('..', 'e'), ('.e', 'm'), ('em', 'm'), ('mm', 'a'), ('ma', '.')]
-----------
..elizabeth.
Trigrams: [('..', 'e'), ('.e', 'l'), ('el', 'i'), ('li', 'z'), ('iz', 'a'), ('za', 'b'), ('ab', 'e'), ('be', 't'), ('et', 'h'), ('th', '.')]
-----------
..minnie.
Trigrams: [('..', 'm'), ('.m', 'i'), ('mi', 'n'), ('in', 'n'), ('nn', 'i'), ('ni', 'e'), ('ie', '.')]
-----------


In [3]:
words = set(words)

In [4]:
from collections import Counter

trigrams = []
for word in words:
    trigrams += [(word[i:i + 2], word[i + 2]) for i in range(len(word) - 2)]

trigram_counts = Counter(trigrams, )

In [5]:
trigrams[:10]

[('..', 'j'),
 ('.j', 'i'),
 ('ji', 'l'),
 ('il', 'l'),
 ('ll', 'a'),
 ('la', 'n'),
 ('an', 'e'),
 ('ne', '.'),
 ('..', 'm'),
 ('.m', 'a')]

In [6]:
trigram_counts.most_common()[:20]

[(('..', 'a'), 9742),
 (('..', 's'), 7729),
 (('..', 'j'), 7632),
 (('..', 'm'), 7258),
 (('..', 'k'), 6973),
 (('..', 'd'), 6450),
 (('..', 't'), 6177),
 (('..', 'c'), 5625),
 (('..', 'l'), 5590),
 (('na', '.'), 5289),
 (('ia', '.'), 4843),
 (('sh', 'a'), 4594),
 (('..', 'r'), 4237),
 (('.m', 'a'), 4002),
 (('ah', '.'), 3839),
 (('.j', 'a'), 3760),
 (('on', '.'), 3554),
 (('..', 'e'), 3500),
 (('..', 'b'), 3362),
 (('..', 'n'), 3322)]

In [7]:
print(len(trigrams))
print(len(trigram_counts))

707197
7985


In [8]:
alphabet = list('.abcdefghijklmnopqrstuvwxyz')
output_stoi = {s: i for i, s in enumerate(alphabet)}
output_itos = {i: s for i, s in enumerate(alphabet)}
# print(output_stoi)
# print(output_itos)

In [9]:
input_stoi = {s: i for i, s in enumerate([j + k for j in alphabet for k in alphabet])}
input_itos = {i: s for i, s in enumerate([j + k for j in alphabet for k in alphabet])}
# print(input_stoi)
# print(input_itos)

#### Sampling from a 2d matrix

In [10]:
import torch

N = torch.zeros(729, 27)
for count in trigram_counts.keys():
    N[input_stoi[count[0]], output_stoi[count[1]]] = trigram_counts[count]

In [11]:
P = (N + 1) / (N + 1).sum(1, keepdim=True)
P

tensor([[1.0648e-05, 1.0374e-01, 3.5809e-02,  ..., 2.6087e-03, 1.5706e-02,
         1.8527e-02],
        [1.0236e-04, 3.4599e-02, 4.0229e-02,  ..., 2.8662e-03, 2.8355e-02,
         2.3851e-02],
        [2.9507e-04, 1.2806e-01, 5.9014e-04,  ..., 2.9507e-04, 7.3768e-03,
         2.9507e-04],
        ...,
        [6.8966e-02, 6.8966e-02, 3.4483e-02,  ..., 3.4483e-02, 3.4483e-02,
         3.4483e-02],
        [2.3770e-01, 1.2022e-01, 8.1967e-03,  ..., 2.7322e-03, 8.1967e-03,
         5.4645e-03],
        [4.6392e-02, 2.8866e-01, 1.0309e-02,  ..., 5.1546e-03, 7.7320e-02,
         5.1546e-03]])

In [12]:
g = torch.Generator().manual_seed(0)
for i in range(10):

    out = []
    ix = '..'
    while True:
        p = P[input_stoi[ix]]
        ox = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        # ox = torch.multinomial(torch.ones(27)/27, num_samples=1, replacement=True, generator=g).item()
        output_char = output_itos[ox]
        out.append(output_char)
        ix = ix[1] + output_char
        if ox == 0:
            break
        
    print(''.join(out))


brona.
kercur.
jer.
li.
lum.
advie.
motah.
suhl.
merlershonia.
aytte.


In [13]:
L = P.log()
nnl = 0
n=0
for w in ['..ahmed.','..rushdi.']:
    for ch1, ch2, ch3 in zip(w[:], w[1:], w[2:]):
        print(f"{ch1+ch2+ch3}: {P[input_stoi[ch1+ch2]][output_stoi[ch3]]:.5f} {(temp := L[input_stoi[ch1+ch2]][output_stoi[ch3]]):.5f} {trigram_counts[(ch1+ch2,ch3)]}")
        nnl -= temp
        n+=1
nnl/n

..a: 0.10374 -2.26585 9742
.ah: 0.01873 -3.97748 182
ahm: 0.03267 -3.42118 179
hme: 0.13312 -2.01653 40
med: 0.02927 -3.53113 79
ed.: 0.14466 -1.93338 194
..r: 0.04513 -3.09831 4237
.ru: 0.06637 -2.71252 282
rus: 0.18220 -1.70266 130
ush: 0.07978 -2.52843 132
shd: 0.00175 -6.34901 13
hdi: 0.15190 -1.88454 11
di.: 0.10726 -2.23249 225


tensor(2.8964)

In [14]:
X_data, y_data = map(list, zip(*trigrams))
X_data[:10], y_data[:10]

(['..', '.j', 'ji', 'il', 'll', 'la', 'an', 'ne', '..', '.m'],
 ['j', 'i', 'l', 'l', 'a', 'n', 'e', '.', 'm', 'a'])

In [15]:
X = torch.tensor([input_stoi[x] for x in X_data])
y = torch.tensor([output_stoi[y] for y in y_data])

In [16]:
nll = -sum(L[x,y] for x,y in zip(X, y))/len(X)

In [17]:
nll

tensor(2.2327)

Ahmed Rushdi is an unlikely name w.r.t. the distribution of this dataset as it has higher Negative Log Likelihood than the dataset average.


#### NN solution

In [18]:
import torch.nn.functional as F
X_enc = F.one_hot(X).float()
W = torch.randn((729,27), requires_grad = True)
y_enc = F.one_hot(y).float()

In [19]:
logits = X_enc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
probs.shape

torch.Size([707197, 27])

In [20]:
nll = -(probs[X, y]).log().mean()
nll

tensor(3.8218, grad_fn=<NegBackward0>)

In [21]:
loss = None
for i in range(1,21):
    logits = X_enc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -(probs[X, y]).log().mean()
    W.grad = None
    loss.backward()
    W.data += -90 * W.grad
    print(f"Epoch {i}: loss\t{loss}")

Epoch 1: loss	3.821833848953247
Epoch 2: loss	3.512383460998535
Epoch 3: loss	3.3989343643188477
Epoch 4: loss	3.293923854827881
Epoch 5: loss	3.231837034225464
Epoch 6: loss	3.16994309425354
Epoch 7: loss	3.1314799785614014
Epoch 8: loss	3.074338912963867
Epoch 9: loss	3.046250581741333
Epoch 10: loss	3.0043814182281494
Epoch 11: loss	2.986933946609497
Epoch 12: loss	2.9497246742248535
Epoch 13: loss	2.937481641769409
Epoch 14: loss	2.9075706005096436
Epoch 15: loss	2.9005300998687744
Epoch 16: loss	2.873008966445923
Epoch 17: loss	2.8687384128570557
Epoch 18: loss	2.8448801040649414
Epoch 19: loss	2.8433432579040527
Epoch 20: loss	2.8209125995635986
